In [4]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 26.6 MB/s 


In [5]:
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 24.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=91625ac6ef4f83504001444ba9ad218d8e5c0a0e9db3d47a06175efa8a7189e9
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [1]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
# list=[10,11,15,16]
list=[15]
df = pd.read_excel("./ja_slide9_data.xlsx", usecols=list,names=['sentence'])
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 133



,sentence
100,競合の製品知識
81,他剤と差別化
114,抗コリンとβ3の作用機序の違いをコンパクトにご紹介すること
42,ベシケアとの差別化からの抗コリンFast確立スキル
9,飽きられない工夫が必要。\nDrのインサイトを得意先別の攻略に活かす事。
49,今月のFFTの様に製品の知識にプラスして営業として必要なスキルや考え方についての研修があれば...
14,疾患・製品知識をさらに強化したい
92,内科の先生はやはり、β3を多く使われてる先生が多い。セカンドチョイスの割合を広げていく点にま...
44,特になし
80,なし


In [2]:
sentences = df.sentence.values
print(sentences[5])

しっかり話し込みが出来た先生には、TOVの効果面や安全性(FORTA等)はお伝えできた。課題は、面会が困難で短時間面会Drや不面Drへの対策が不十分なこと。


In [3]:
col15_sentence=[]

for sen in sentences:
    col15_sentence.append(sen)
col15_sentence_text=''.join(col15_sentence)
len(col15_sentence_text)
# type(col15_sentence)
# len(col15_sentence)

3186

In [4]:
import MeCab
import ipadic
sentence=col15_sentence_text
# sentence = "AWSの有名なサービスにAmazon Elastic Compute Cloud (EC2) とAmazon Simple Storage Service (S3) がある。これまでのクライアントが保有していた物理的なサーバファームと比較してAWSは大規模な計算処理能力を速やかに提供出来ることが強みである。"
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)
word_list=tagger.parse(sentence)
# print(word_list)
# type(word_list) str

In [5]:
# sentence = "AWSの有名なサービスにAmazon Elastic Compute Cloud (EC2) とAmazon Simple Storage Service (S3) がある。これまでのクライアントが保有していた物理的なサーバファームと比較してAWSは大規模な計算処理能力を速やかに提供出来ることが強みである。"
# sentence = sentences[5]
m = tagger.parseToNode(sentence)

keywords = []

while m:
    if m.feature.split(',')[0] == '名詞':
        keywords.append(m.surface)
    m = m.next

print(keywords)

['ベタニスベオーバ', '剤', '理解', 'TOV', '継続', '訴求', '不安', 'イン', 'サイト', '深堀', '重要', 'こと', '着任', '時', '重要', '指導', '現場', 'MR', '今回', 'FFT', 'イン', 'サイト', '深堀', '重要', 'の', 'MR', 'これ', '現場', 'トレーニング', '必要', '先生', 'TOV', '効果', '面', '安全', '性', 'FORTA', '等', '課題', '面会', '困難', '短時間', '面会', 'Dr', '面', 'Dr', '対策', '不十分', 'こと', '3', '一', '選択', 'コリン', '二', '選択', 'OABSS', '排尿', '日誌', '診療', '時間', '問診', '切迫', '性', '尿', '失禁', '有無', '先生', '先生', 'トビ', 'エース', '服用', '患者', 'さん', '像', '一言', 'よう', 'フレーズ', '安全', '性', '質問', 'オブジェクションハンドリング', 'おむつ', 'ケア', '知識', '工夫', '必要', 'Dr', 'イン', 'サイト', '得意', '先', '別', '攻略', '事', '苦手', 'イン', 'サイト', '把握', 'スキル', '身', 'Dr', 'ニーズ', '把握', '位置付け', '確立', 'コリン', '薬', '一', '選択', '処方', '患者', '像', '具体', '的', '提示', 'スライド', 'データ', '疾患', '製品', '知識', '強化', 'TOV', '臨床', '試験', '豊富', '臨床', '試験', '部分', '定期', '的', '復習', 'TOV', 'UUI', '試験', '明確', '疾患', 'スキル', 'GE', '浸食', 'こと', '国策', '抵抗', '事', 'の', 'TOV', '処方', '経験', 'Dｒ', 'コリン', '剤', 'なか', '位置づけ', '場合', '競合', '品', '情報', 'コリン', '薬', 'B', '3', '受容', '体',

In [6]:
print(len(keywords))

713


In [7]:
def get_stopword():
    s=set()
    with open('jp_stopwords.txt','r',encoding='UTF-8') as f:
        for line in f:
            s.add(line.strip())
    return s
#去停用词
def rm_stopwords(text,stopword_list):
    s=stopword_list
    # text_tokens=text.split()
    text_list=[]
    #remove stopwords
    for word in text:
        if word not in s:
            text_list.append(word)
    return text_list
# 去掉文本中的停用词
def drop_stopwords(words):
    import itertools
    content_dropped=[word for word in words if word not in stopword]
    content=''.join(list(itertools.chain(*content_dropped)))
    return content

stopword=get_stopword()
contents=rm_stopwords(keywords,stopword)
contents

['ベタニスベオーバ',
 '剤',
 '理解',
 'TOV',
 '継続',
 '訴求',
 '不安',
 'イン',
 'サイト',
 '深堀',
 '重要',
 'こと',
 '着任',
 '時',
 '重要',
 '指導',
 '現場',
 'MR',
 '今回',
 'FFT',
 'イン',
 'サイト',
 '深堀',
 '重要',
 'MR',
 '現場',
 'トレーニング',
 '必要',
 '先生',
 'TOV',
 '効果',
 '面',
 '安全',
 '性',
 'FORTA',
 '等',
 '課題',
 '面会',
 '困難',
 '短時間',
 '面会',
 'Dr',
 '面',
 'Dr',
 '対策',
 '不十分',
 'こと',
 '3',
 '一',
 '選択',
 'コリン',
 '二',
 '選択',
 'OABSS',
 '排尿',
 '日誌',
 '診療',
 '時間',
 '問診',
 '切迫',
 '性',
 '尿',
 '失禁',
 '有無',
 '先生',
 '先生',
 'トビ',
 'エース',
 '服用',
 '患者',
 'さん',
 '像',
 '一言',
 'よう',
 'フレーズ',
 '安全',
 '性',
 '質問',
 'オブジェクションハンドリング',
 'おむつ',
 'ケア',
 '知識',
 '工夫',
 '必要',
 'Dr',
 'イン',
 'サイト',
 '得意',
 '先',
 '別',
 '攻略',
 '事',
 '苦手',
 'イン',
 'サイト',
 '把握',
 'スキル',
 '身',
 'Dr',
 'ニーズ',
 '把握',
 '位置付け',
 '確立',
 'コリン',
 '薬',
 '一',
 '選択',
 '処方',
 '患者',
 '像',
 '具体',
 '的',
 '提示',
 'スライド',
 'データ',
 '疾患',
 '製品',
 '知識',
 '強化',
 'TOV',
 '臨床',
 '試験',
 '豊富',
 '臨床',
 '試験',
 '部分',
 '定期',
 '的',
 '復習',
 'TOV',
 'UUI',
 '試験',
 '明確',
 '疾患',
 'スキル',
 'GE',
 '浸食',
 'こと',
 '

In [8]:
print(len(contents))

704


### 词频

In [17]:
from collections import Counter
import matplotlib.pyplot as plt
from pylab import *
mpl.rcParams['font.sans-serif']=['SimHei']

def word_freq(fenci_con,n=5):#n表示出现次数最多的前n项
    c=Counter(fenci_con)
    d=dict(c.most_common(n))
    # print(d)
    # plt.figure(figsize=(15,5))
    # plt.bar(d.keys(),d.values())
    return d

In [19]:
most_common_words=word_freq(contents,n=20)
print(most_common_words)

{'知識': 18, '薬': 15, 'TOV': 14, 'イン': 13, 'サイト': 13, '競合': 13, 'コリン': 12, '強化': 12, 'スキル': 11, 'データ': 11, '品': 11, '情報': 11, 'Dr': 10, '3': 10, '製品': 10, 'DTL': 10, '差別': 9, '化': 9, 'こと': 8, '先生': 8}


In [21]:
import jieba
import matplotlib.pyplot as plt
from wordcloud import WordCloud
def word_cloud(text_list):

    # WordCloud
    wc = WordCloud(max_words=2000,
                max_font_size=40,
                font_path='./fonts/simhei.ttf',
                background_color='white',
                random_state=42,
                relative_scaling=0)

    wc.generate(text_list)
    # Plot
    plt.figure()
    plt.axis('off')
    plt.imshow(wc)
    plt.show()


In [1]:
#英文demo
word_cloud(contents)

NameError: ignored

In [ ]:

m = tagger.parseToNode(contents)

keywords_rm_stopwords = []

while m:
    if m.feature.split(',')[0] == '名詞':
        keywords.append(m.surface)
    m = m.next

print(keywords_rm_stopwords)

[]


In [ ]:
print(len(keywords_rm_stopwords))

0


In [ ]:
import MeCab

sentence = "AWSの有名なサービスにAmazon Elastic Compute Cloud (EC2) とAmazon Simple Storage Service (S3) がある。これまでのクライアントが保有していた物理的なサーバファームと比較してAWSは大規模な計算処理能力を速やかに提供出来ることが強みである。"
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)
t = MeCab.Tagger(ipadic.MECAB_ARGS)
t.parse('')
m = t.parseToNode(sentence)

keywords = []

while m:
    if m.feature.split(',')[0] == '名詞':
        keywords.append(m.surface)
    m = m.next

print(keywords)

NameError: ignored

In [ ]:
import MeCab
import ipadic
tagger = MeCab.Tagger('-Owakati')

print(tagger.parse("図書館にいた事がバレた"))

RuntimeError: ignored

In [ ]:
help(MeCab.Tagger)

Help on class Tagger in module MeCab:

class Tagger(Tagger)
 |  Tagger(rawargs='')
 |  
 |  MeCab::Tagger
 |  
 |  Method resolution order:
 |      Tagger
 |      Tagger
 |      builtins.SwigPyObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, rawargs='')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Tagger:
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 | 